<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/Sectors_Support_and_Volatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df_nse_sectors = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/NSE_Sectors_Indices.csv')
exclude = ['PVRINOX', 'JIOFIN', 'RENUKA', 'BAJAJHIND', 'TV18BRDCST', 'NETWORK18', 'TATASTEEL']

df_nse_sectors = df_nse_sectors[~df_nse_sectors['Symbol'].isin(exclude)]
#df_nse_sectors = df_nse_sectors.tail()

In [2]:
df_nse_sectors

,Sector,Symbol,Weight
0,AUTO,TATAMOTORS,17.25
1,AUTO,M&M,16.38
2,AUTO,MARUTI,14.90
3,AUTO,BAJAJ-AUTO,9.61
4,AUTO,HEROMOTOCO,6.65
...,...,...,...
93,FOOTWEAR,CAMPUS,14.20
94,FOOTWEAR,MAYURUNIQ,3.70
95,SUGAR,EIDPARRY,28.80
97,SUGAR,BALRAMCHIN,26.90


In [3]:
import yfinance as yf
import numpy as np


# Function to get current PE ratio, market cap, and historical prices
def get_stock_data(stock):
    ticker = yf.Ticker(stock + '.NS')
    pe_ratio = ticker.info['trailingPE']
    market_cap = ticker.info['marketCap']
    historical_prices = ticker.history(period='5y', interval='1d')['Close']
    historical_pe_ratios = historical_prices / ticker.info['trailingEps']
    return pe_ratio, market_cap, historical_prices, historical_pe_ratios

# Dataframes to store stock data
stock_data = []
historical_data = {}

for _, row in df_nse_sectors.iterrows():
    sector = row['Sector']
    stock = row['Symbol']
    print(stock)
    pe_ratio, market_cap, historical_prices, historical_pe_ratios = get_stock_data(stock)
    stock_data.append({'Stock': stock, 'Sector': sector, 'PE Ratio': pe_ratio, 'Market Cap': market_cap})
    historical_data[stock] = {'Prices': historical_prices, 'PE Ratios': historical_pe_ratios}

df = pd.DataFrame(stock_data)


TATAMOTORS
M&M
MARUTI
BAJAJ-AUTO
HEROMOTOCO
TVSMOTOR
HDFCBANK
ICICIBANK
SBIN
AXISBANK
KOTAKBANK
BAJFINANCE
BAJAJFINSV
IRFC
PFC
RECLTD
INFY
TCS
HCLTECH
TECHM
WIPRO
LTIM
ITC
HINDUNILVR
NESTLEIND
TATACONSUM
VBL
BRITANNIA
SUNPHARMA
DRREDDY
CIPLA
DIVISLAB
LUPIN
AUROPHARMA
TORNTPHARM
ZEEL
SUNTV
NAZARA
HINDALCO
ADANIENT
JSWSTEEL
VEDL
JINDALSTEL
RELIANCE
LT
IOC
ONGC
NTPC
POWERGRID
TATAPOWER
ULTRACEMCO
ADANIPORTS
BHARTIARTL
APOLLOHOSP
PIDILITIND
SRF
LINDEINDIA
FLUOROCHEM
DEEPAKNTR
GODREJIND
TATACHEM
AARTIIND
CASTROLIND
ULTRACEMCO
AMBUJACEM
SHREECEM
ACC
DALBHARAT
HAVELLS
DIXON
VOLTAS
BLUESTARCO
CROMPTON
KAYNES


ERROR:yfinance:KAYNES.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', 'ytd', 'max']


WHIRLPOOL
AMBER
TITAN
KALYANKJIL
RAJESHEXPO
VAIBHAVGBL
TRIDENT
WELSPUNLIV
RAYMOND
GARFIBRES
JINDWORLD
METROBRAND
RELAXO
BATAINDIA
CAMPUS
MAYURUNIQ
EIDPARRY
BALRAMCHIN
TRIVENI


In [4]:
# Calculate sector indices and weighted PE ratios
sector_metrics = []

for sector in df['Sector'].unique():
    sector_df = df[df['Sector'] == sector].nlargest(5, 'Market Cap')

    # Calculate weighted average prices for sector index
    sector_prices = pd.DataFrame({stock: historical_data[stock]['Prices'] for stock in sector_df['Stock']})
    sector_weights = sector_df['Market Cap'] / sector_df['Market Cap'].sum()
    sector_index = (sector_prices * sector_weights.values).sum(axis=1)

    # Calculate 200-day SMA for sector index
    sma_200 = sector_index.rolling(window=200).mean().iloc[-1]

    # Calculate weighted PE ratio
    weighted_pe = np.average(sector_df['PE Ratio'], weights=sector_df['Market Cap'])

    # Calculate median PE ratio over the last 5 years
    sector_pe_ratios = pd.DataFrame({stock: historical_data[stock]['PE Ratios'] for stock in sector_df['Stock']})
    sector_weighted_pe = (sector_pe_ratios * sector_weights.values).sum(axis=1)
    median_pe = sector_weighted_pe.median()

    # Get latest sector index value
    latest_sector_index = sector_index.iloc[-1]

    # Append sector metrics to list
    sector_metrics.append({
        'Sector_Name': sector,
        'Sector_Index': round(latest_sector_index, 0),
        '200_SMA_Sector_Index': round(sma_200, 0),
        'Sector_PE': round(weighted_pe, 1),
        'Sector_PE_5Y_Median': round(median_pe, 1)
    })

# Create DataFrame for sector metrics
sector_df = pd.DataFrame(sector_metrics)

sector_df


,Sector_Name,Sector_Index,200_SMA_Sector_Index,Sector_PE,Sector_PE_5Y_Median
0,AUTO,6182.0,5256.0,28.3,11.0
1,BANKS,1343.0,1227.0,16.5,12.3
2,FINANCE,2724.0,2784.0,25.9,15.2
3,IT,3074.0,2736.0,30.4,22.3
4,FMCG,2032.0,1848.0,56.0,34.5
5,PHARMA,3045.0,2494.0,45.1,25.9
6,MEDIA,722.0,573.0,34.5,29.6
7,METAL,1610.0,1478.0,51.2,28.5
8,INFRA,2252.0,2032.0,45.6,24.9
9,CHEMICAL,3918.0,3440.0,93.0,58.6


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

def fetch_data(ticker, period='5y'):
    stock = yf.Ticker(ticker+ '.NS')
    data = stock.history(period=period, interval='1wk')
    return data

def find_support_range(data, range_width=5):
    # Define the range of prices to analyze
    min_price = data['Close'].min()
    max_price = data['Close'].max()
    # Create bins for the price ranges
    bins = np.arange(min_price, max_price, range_width)
    # Count occurrences of closing prices in each bin, no need to pre-specify labels
    data['Price_Range'] = pd.cut(data['Close'], bins=bins, include_lowest=True)
    support_range = data['Price_Range'].value_counts().idxmax()
    return support_range.left, support_range.right

def suggest_support_range(ticker, period='5y', range_width=5):
    data = fetch_data(ticker, period)
    support_range = find_support_range(data, range_width)
    return support_range

def calculate_volatility(data):
    # Calculate weekly returns
    data['Weekly_Return'] = data['Close'].pct_change()

    # Calculate the standard deviation of weekly returns
    volatility = data['Weekly_Return'].std() * np.sqrt(52)  # Annualize the volatility
    return volatility

def calculate_risk_reward_ratio(ticker, period='5y', range_width=5):
    data = fetch_data(ticker, period)

    # Get support range
    support_lower, support_upper = suggest_support_range(ticker, period, range_width)
    support = (support_lower + support_upper) / 2  # Using midpoint of the support range

    # Get all-time high (ATH) and current market price (CMP)
    ATH = data['Close'].max()
    CMP = data['Close'].iloc[-1]

    # Calculate Risk-Reward Ratio (RRR)
    RRR = (CMP - support) / (ATH - CMP)

    # Calculate Volatility
    volatility = calculate_volatility(data)

    return {
        'Ticker': ticker,
        'Support': round(support,0),
        'ATH': round(ATH,0),
        'CMP': round(CMP,0),
        'RRR': round(RRR,2),
        'Volatility': round(volatility,2)
    }

def get_features_for_tickers(ticker_list, period='5y', range_width=5):
    features = []

    for ticker in ticker_list:
        try:
            feature = calculate_risk_reward_ratio(ticker, period, range_width)
            features.append(feature)
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    df = pd.DataFrame(features)
    df_sorted = df.sort_values(by='RRR')

    return df_sorted

mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
tickers = mypps_df[mypps_df['InFolio'] == 0]['Symbol']
df_features = get_features_for_tickers(tickers)
df_features

ERROR:yfinance:ARE&M.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:ARE&M.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']


Error processing ARE&M: arange: cannot compute length


<ipython-input-5-e1fbe72f731e>:46: RuntimeWarning: divide by zero encountered in scalar divide
  RRR = (CMP - support) / (ATH - CMP)
<ipython-input-5-e1fbe72f731e>:46: RuntimeWarning: divide by zero encountered in scalar divide
  RRR = (CMP - support) / (ATH - CMP)
<ipython-input-5-e1fbe72f731e>:46: RuntimeWarning: divide by zero encountered in scalar divide
  RRR = (CMP - support) / (ATH - CMP)
<ipython-input-5-e1fbe72f731e>:46: RuntimeWarning: divide by zero encountered in scalar divide
  RRR = (CMP - support) / (ATH - CMP)
<ipython-input-5-e1fbe72f731e>:46: RuntimeWarning: divide by zero encountered in scalar divide
  RRR = (CMP - support) / (ATH - CMP)
<ipython-input-5-e1fbe72f731e>:46: RuntimeWarning: divide by zero encountered in scalar divide
  RRR = (CMP - support) / (ATH - CMP)
ERROR:yfinance:JIOFIN.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:JIOFIN.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo'

Error processing JIOFIN: arange: cannot compute length


ERROR:yfinance:SPORTKING.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', 'ytd', 'max']
ERROR:yfinance:SPORTKING.NS: Period '5y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', 'ytd', 'max']


Error processing SPORTKING: arange: cannot compute length


/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Ticker,Support,ATH,CMP,RRR,Volatility
34,WHIRLPOOL,2161.0,2643.0,2131.0,-0.06,0.29
11,CUB,135.0,239.0,167.0,0.45,0.36
2,APEX,216.0,402.0,284.0,0.58,0.50
30,SRF,2309.0,2726.0,2523.0,1.05,0.30
28,SAIL,81.0,170.0,146.0,2.81,0.50
25,NMDC,109.0,280.0,236.0,2.87,0.42
21,J&KBANK,28.0,144.0,117.0,3.27,0.55
12,DMART,3498.0,5324.0,4910.0,3.41,0.34
27,RELIANCE,2245.0,3193.0,2999.0,3.87,0.27
10,CROMPTON,290.0,475.0,438.0,3.92,0.32
